# Data Sourcing

> Pull data from Citi Velocity

In [ ]:
#| default_exp data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import http.client 
import ssl
import json
import ast
import pandas as pd

In [ ]:
#| export
def get_cv_token(id='', secret=''):
    conn = http.client.HTTPSConnection("api.citivelocity.com")
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        "Accept": "application/json",
        }
    payload = "client_id="+id+"&client_secret="+secret+"&grant_type=client_credentials&scope=/api"
    conn.request("POST", "/markets/cv/api/oauth2/token", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return data.decode("utf-8")


In [ ]:
#| export
def get_cv_data(id, token, payload):
    conn = http.client.HTTPSConnection("api.citivelocity.com")
    token = ast.literal_eval(token)
    payload = json.dumps(payload)
    headers = {
        'authorization': "Bearer "+token['access_token'],
        'accept': "application/json",
        'content-type': "application/json"
        }
    conn.request("POST", "/markets/analytics/chartingbe/rest/external/authed/data?client_id=" + id, payload, headers)
    res = conn.getresponse()
    data = res.read()
    return data.decode("utf-8")

In [ ]:
#| export
def get_cv_data_df(id, token, payload):
    data = get_cv_data(id, token, payload)
    data = ast.literal_eval(data)

    # Extract the required data from the dictionary
    indicator_data = data['body']
    indicators = list(indicator_data.keys())

    # Create a DataFrame with the required columns and index
    df = pd.DataFrame(index=indicator_data[indicators[0]]['x'])

    # Add indicator columns to the DataFrame
    for indicator in indicators:
        df[indicator] = indicator_data[indicator]['c']

    # Rename the index column to 'Date'
    df.index.name = 'Date'
    df.index = pd.to_datetime(df.index, format='%Y%m%d')

    return df


In [ ]:
#| hide
id = '08345866-fcf9-45d9-a4c6-cad82e15a95c'
secret = 'G8rK4oQ1cB3nM6vA8rV7sB1bK4oS0sS1vW0bJ4iW1kN8mL2aS8'
token = get_cv_token(id, secret)

payload = {
    "startDate": 20100101,
    "endDate": 20101231,
    "tags":[
    "COMMODITIES.SPOT.SPOT_GOLD",
    "FX.SPOT.EUR.USD.CITI",
    ],
    "frequency":"DAILY"
}
data = get_cv_data_df(id, token, payload)

In [ ]:
data

,COMMODITIES.SPOT.SPOT_GOLD,FX.SPOT.EUR.USD.CITI
Date,,
2010-01-01,1097.35,1.43260
2010-01-04,1120.40,1.44380
2010-01-05,1119.05,1.44145
2010-01-06,1138.90,1.43885
2010-01-07,1132.30,1.43275
...,...,...
2010-12-27,1384.10,1.31485
2010-12-28,1406.39,1.31305
2010-12-29,1410.95,1.31345


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()